<a href="https://www.kaggle.com/code/iqmansingh/cnn-dogs-vs-cats?scriptVersionId=135140893" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# Importing Libraries 

In [ ]:
import numpy as np
import pandas as pd 
import tensorflow as tf
import datetime
import warnings
warnings.filterwarnings("ignore")
pd.plotting.register_matplotlib_converters()
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
plt.style.use('dark_background')

# Defining Classes & Generators

In [ ]:
classes = ['cats','dogs']

In [ ]:
generator = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255.0, 
                               shear_range=0.15, 
                               zoom_range=0.2, 
                               horizontal_flip=True
                              ) 

train_data = generator.flow_from_directory( directory='../input/cat-and-dog/training_set/training_set', 
                                            target_size=(128, 128),
                                            color_mode='grayscale',
                                            classes=classes, 
                                            batch_size=64, 
                                            )


test_data = generator.flow_from_directory( directory='../input/cat-and-dog/test_set/test_set', 
                                           target_size=(128, 128), 
                                           color_mode='grayscale',
                                           classes=classes, 
                                           batch_size=64,
                                           shuffle=False
                                           )

# Building the CNN

In [ ]:
model = tf.keras.models.Sequential()

In [ ]:
model.add(tf.keras.layers.Conv2D(filters=32,kernel_size=3,activation='relu',padding='same',input_shape=[128,128,1]))

In [ ]:
model.add(tf.keras.layers.Conv2D(filters=32,kernel_size=3,activation='relu',padding='same'))

In [ ]:
model.add(tf.keras.layers.MaxPool2D(pool_size=2,strides=2,padding='valid'))

In [ ]:
model.add(tf.keras.layers.BatchNormalization())

In [ ]:
model.add(tf.keras.layers.Conv2D(filters = 64,kernel_size = 3,padding = 'same',activation = 'relu'))

In [ ]:
model.add(tf.keras.layers.Conv2D(filters = 64,kernel_size = 3,padding = 'same',activation = 'relu'))

In [ ]:
model.add(tf.keras.layers.MaxPool2D(pool_size=2,strides=2,padding='valid'))

In [ ]:
model.add(tf.keras.layers.BatchNormalization())

In [ ]:
model.add(tf.keras.layers.Flatten())

In [ ]:
model.add(tf.keras.layers.Dense(units=128,activation='relu'))

In [ ]:
model.add(tf.keras.layers.Dropout(0.3))

In [ ]:
model.add(tf.keras.layers.Dense(units=128,activation='relu'))

In [ ]:
model.add(tf.keras.layers.Dense(units=2,activation='softmax'))

In [ ]:
model.summary()

In [ ]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['categorical_crossentropy','accuracy'])

# Training the Model

In [ ]:
earlyStopping =  tf.keras.callbacks.EarlyStopping(patience=3, monitor='val_accuracy',restore_best_weights=True,start_from_epoch=25)

history = model.fit(train_data, epochs=50, validation_data=test_data, shuffle=True, callbacks=[earlyStopping],batch_size=64)
# history = model.fit(train_data, epochs=10, validation_split=0.2, shuffle=True, callbacks=[earlyStopping],batch_size=32)


In [ ]:
df = pd.DataFrame(history.history)
df_loss = df[['loss', 'val_loss']]
df_acc =  df[['accuracy', 'val_accuracy']]

# Plotting Validation Loss & Accuracy

In [ ]:
plt.figure(figsize=(10,4))
plt.title("Loss & Val Loss")
sns.lineplot(df_loss,palette="flare");

plt.title("Accuracy & Val Accuracy")
plt.figure(figsize=(10,4))
sns.lineplot(df_acc,palette="flare");

# Evaluating Test Set

In [ ]:
evals = model.evaluate(test_data)
print(evals)

In [ ]:
print("Test Loss:", evals[0])
print("Test Accuracy:", evals[2])